# **Introduction**

<div class="align-center">
  <a href="https://getindexify.ai/"><img src="https://getindexify.ai/images/logos/base2.svg" width="145"></a>
  <a href="https://discord.com/invite/kF8UZACA7r"><img src="https://raw.githubusercontent.com/rishiraj/random/main/Discord%20button.png" width="145"></a><br>
  Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/tensorlakeai/indexify">Github</a></i> ⭐
</div>

This notebook demonstrates how Indexify can make it easier to quickly extract insights from complex SEC filings like the Form 10-K annual report. Using Uber's 10-K as an example, we show how the Indexify library can enable question answering on the filing text to get rapid answers. We also illustrate how schema-based extraction can pull key data points from the unstructured document. The combination of question answering and schema-based extraction provides a powerful toolkit to derive insights from dense financial filings.

## **Setup**

In [ ]:
# %pip install indexify indexify-extractor-sdk

# # Download Indexify Server
# !curl https://getindexify.ai | sh

# # Install Poppler (required for PDF extraction)
# # You can use brew on MacOS.
# !sudo apt-get install -y poppler-utils

# # Download Extractors
# !indexify-extractor download tensorlake/chunk-extractor
# !indexify-extractor download tensorlake/minilm-l6
# !indexify-extractor download tensorlake/pdfextractor

After installing the necessary libraries, download the server, and the extractors, you need to restart the runtime. Then, you have to run Indexify Server with the Extractors.

Open 2 terminals and run the following commands:

```bash
# Terminal 1
./indexify server -d

# Terminal 2
indexify-extractor join-server
```

## **Test the extractors**

We will try PDFExtractor first. The PDFExtractor can extract all the values from text as well as tables in one shot and passes it to the next chained extractors which can be used for question answering.

We'll start by downloading Uber's Form 10-K report.

In [ ]:
import requests
req = requests.get("https://d18rn0p25nwr6d.cloudfront.net/CIK-0001543151/6fabd79a-baa9-4b08-84fe-deab4ef8415f.pdf")

with open('form10-k.pdf','wb') as f:
    f.write(req.content)

In [2]:
cd ..

/home/quamer23nasim38/faster-patient-onboarding-from-medical-history-using-indexify


/home/quamer23nasim38/venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
load_indexify_extractors("custom_extractor:MyExtractor")

ValueError: Invalid directory: /home/quamer23nasim38/faster-patient-onboarding-from-medical-history-using-indexify/indexify-extractor-template-main/custom_extractor:MyExtractor

In [ ]:
load_indexify_extractors("/home/quamer23nasim38/faster-patient-onboarding-from-medical-history-using-indexify/indexify-extractor-template-main/custom_extractor.py")

In [1]:
from indexify_extractor_sdk import load_indexify_extractors, Content

pdfextractor, pdfconfig_cls = load_indexify_extractors("indexify_extractors.pdfextractor.pdf_extractor:PDFExtractor")
content = Content.from_file("form10-k.pdf")
config = pdfconfig_cls()

pdf_result = pdfextractor.extract(content, config)
text_content = next(content.data.decode('utf-8') for content in pdf_result if content.content_type == 'text/plain')

ValueError: Invalid directory: /home/quamer23nasim38/faster-patient-onboarding-from-medical-history-using-indexify/notebooks/indexify_extractors.pdfextractor.pdf_extractor:PDFExtractor

In [ ]:
print(text_content)

## **Create a Client**
Instantiate the Indexify Client

In [1]:
from indexify import IndexifyClient
client = IndexifyClient()

## **1. Question Answering Task**

### **Extraction Graph Setup**

1. Import the `ExtractionGraph` class from the `indexify` package.

2. Define the extraction graph specification in YAML format:
   - Set the name of the extraction graph to "pdfqa".
   - Define the extraction policies:
     - Use the "tensorlake/pdfextractor" extractor for PDF marking and name it "docextractor".
     - Use the "tensorlake/chunk-extractor" for text chunking and name it "chunks".
       - Set the input parameters for the chunker:
         - `chunk_size`: 1000 (size of each text chunk)
         - `overlap`: 100 (overlap between chunks)
         - `content_source`: "docextractor" (source of content for chunking)
     - Use the "tensorlake/minilm-l6" extractor for embedding and name it "get-embeddings".
       - Set the content source for embedding to "chunks".

3. Create an `ExtractionGraph` object from the YAML specification using `ExtractionGraph.from_yaml()`.

4. Create the extraction graph on the Indexify client using `client.create_extraction_graph()`.

In [2]:
from indexify import ExtractionGraph

extraction_graph_spec = """
name: 'pdfqa2'
extraction_policies:
   - extractor: 'tensorlake/pdfextractor'
     name: 'docextractor'
   - extractor: 'tensorlake/chunk-extractor'
     name: 'chunker'
     input_params:
        chunk_size: 1000
        overlap: 100
     content_source: 'docextractor'
   - extractor: 'tensorlake/minilm-l6'
     name: 'embedder'
     content_source: 'chunker'
"""

extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)

### **Upload the FORM 10-K PDF File**

In [3]:
content_id = client.upload_file("pdfqa2", "form10-k_.pdf")
client.wait_for_extraction(content_id)  

Waiting for extraction to complete for content id:  a3cdb41598978a01
Extraction completed for content id:  a3cdb41598978a01


### **What is happening behind the scenes**

Indexify is designed to seamlessly respond to ingestion events by assessing all existing policies and triggering the necessary extractors for extraction. Once the PDF extractor completes the process of extracting texts, bytes, and JSONs from the document, it automatically initiates the embedding extractor to chunk the content, extract embeddings, and populate an index.

With Indexify, you have the ability to upload hundreds of PDF files simultaneously, and the platform will efficiently handle the extraction and indexing of the contents without requiring manual intervention. To expedite the extraction process, you can deploy multiple instances of the extractors, and Indexify's built-in scheduler will transparently distribute the workload among them, ensuring optimal performance and efficiency.

### **Perform RAG with OpenAI**

In [4]:
def get_context(question: str, index: str, top_k=3):
    results = client.search_index(name=index, query=question, top_k=top_k)
    context = ""
    for result in results:
        context = context + f"content id: {result['content_id']} \n\n passage: {result['text']}\n"
    return context

In [5]:
question = "What are the disclosure with respect to Foreign Subsidiaries?"
context = get_context(question, "pdfqa.embedder.embedding")
context

ApiException: status: Aborted, message: "Index with id 352c626f03d56331 not found", details: [], metadata: MetadataMap { headers: {"content-type": "application/grpc", "date": "Thu, 22 Aug 2024 07:55:48 GMT", "content-length": "0"} }

In [ ]:
def create_prompt(question, context):
    return f"Answer the question, based on the context.\n question: {question} \n context: {context}"

prompt = create_prompt(question, context)

In [ ]:
from openai import OpenAI
client_openai = OpenAI()

Now ask any question related to the ingested FORM 10-K PDF document

In [ ]:
chat_completion = client_openai.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
)
print(chat_completion.choices[0].message.content)

## **2. Schema-based Retrieval Task**
Alert: The following example will cost a lot of OpenAI credits. Move ahead at your own risk.

### **Extraction Graph Setup**

In [7]:
extraction_graph_spec = """
name: 'pdfschema3'
extraction_policies:
   - extractor: 'tensorlake/pdfextractor'
     name: 'docextractor'
   - extractor: 'tensorlake/schema'
     name: 'schemaprocessor'
     input_params:
        service: 'openai'
        schema: {'properties': {'file_number': {'title': 'File Number', 'type': 'string'}, 'registrant_name': {'title': 'Registrant Name', 'type': 'string'}, 'jurisdiction': {'title': 'Jurisdiction', 'type': 'string'}, 'employer_id_number': {'title': 'Employer Id Number', 'type': 'string'}, 'address': {'title': 'Address', 'type': 'string'}, 'telephone_number': {'title': 'Telephone Number', 'type': 'string'}, 'title_of_each_class': {'title': 'Title Of Each Class', 'type': 'string'}, 'trading_symbol': {'title': 'Trading Symbol', 'type': 'string'}, 'name_of_exchange': {'title': 'Name Of Exchange', 'type': 'string'}}, 'required': ['file_number', 'registrant_name', 'jurisdiction', 'employer_id_number', 'address', 'telephone_number', 'title_of_each_class', 'trading_symbol', 'name_of_exchange'], 'title': 'Form', 'type': 'object'}
     content_source: 'docextractor'
"""

extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)

### **Upload the FORM 10-K PDF File**

In [8]:
content_id = client.upload_file("pdfschema3", "form10-k_.pdf")
client.wait_for_extraction(content_id)  

Waiting for extraction to complete for content id:  bb896dc28537dea2
Extraction completed for content id:  bb896dc28537dea2


### **View the extracted content**

In [9]:
client.get_extracted_content(content_id, 'pdfschema3', 'schemaprocessor')

[{'id': '571abd40144c1f67',
  'mime_type': 'text/plain',
  'content': b'{"name":"Uber Technologies, Inc."}'}]